In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("object-detection", model="facebook/detr-resnet-50")

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
from PIL import Image

In [4]:
raw_image = Image.open('../data/town.jpeg')
# resized_image = raw_image.resize((569, 491))


In [9]:
import numpy as np

#we need to convert the image into a numpy array to use the opencv
image_np = np.array(raw_image)

In [6]:
pipeline_output = pipe(raw_image)

In [7]:
import cv2

def render_output(image, detections):
    # Loop through each detection
    for detection in detections:
        # Get the label and bounding box coordinates
        label = detection['label']
        box = detection['box']
        xmin, ymin, xmax, ymax = box['xmin'], box['ymin'], box['xmax'], box['ymax']

        # Draw the bounding box rectangle
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

        # Put the label text above the bounding box
        cv2.putText(image, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the rendered image
    cv2.imshow('Object Detection Output', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [8]:
render_output(image_np, pipeline_output)

In [1]:
import os
import gradio as gr

In [6]:
def get_pipeline_prediction(pil_image):

    image_np = np.array(pil_image)
    
    pipeline_output = pipe(image_np)
    
    processed_image = render_output(image_np, pipeline_output)
    return processed_image

In [8]:
demo = gr.Interface(
  fn=get_pipeline_prediction,
  inputs=gr.Image(label="Input image", 
                  type="pil"),
  outputs=gr.Image(label="Output image with predicted instances",
                   type="pil")
)

In [9]:
demo.launch(share=False)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/tselentis/Desktop/APP/.env/lib/python3.11/site-packages/gradio/queueing.py", line 522, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tselentis/Desktop/APP/.env/lib/python3.11/site-packages/gradio/route_utils.py", line 260, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tselentis/Desktop/APP/.env/lib/python3.11/site-packages/gradio/blocks.py", line 1741, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tselentis/Desktop/APP/.env/lib/python3.11/site-packages/gradio/blocks.py", line 1296, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tselentis/Desktop/APP/.env/lib/python3.11/site-packages/anyio/to_thread.py", line 56, in run_sy

In [10]:
demo.close()

Closing server running on port: 7860


TODO fix the output in gradio